In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import torch

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [2]:
from keras.callbacks import ModelCheckpoint

In [3]:
# # Train Data
# df_train_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-train.csv', sep='\t', header=None)
# df_train_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-train.csv', sep='\t', header=None)

# # Dev Data
# df_dev_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-dev.csv', sep='\t', header=None)
# df_dev_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-dev.csv', sep='\t', header=None)

# # Test Data
# df_test_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-test.csv', sep='\t', header=None)
# df_test_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-test.csv', sep='\t', header=None)

In [4]:
df_train_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_B_train.tsv', sep = '\t')
df_dev_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_B_dev.tsv', sep = '\t' )
df_test_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_B_test.csv', sep = '\t')


In [5]:
df_train_ea.rename(columns = {'text':1, 'category':0}, inplace = True)
df_dev_ea.rename(columns = {'text':1, 'category':0}, inplace = True)
df_test_ea.rename(columns = {'text':1, 'category':0}, inplace = True)

In [6]:
train_data = df_train_ea.dropna()
val_data = df_dev_ea.dropna()
test_data = df_test_ea.dropna()


In [7]:
train_data.drop([1184,1922], inplace = True)

In [8]:
#train_data[0].value_counts()

In [9]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 64000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 128
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(train_data[1])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 74190 unique tokens.


In [10]:
X_train = tokenizer.texts_to_sequences(train_data[1])
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

X_val = tokenizer.texts_to_sequences(val_data[1])
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_val.shape)

X_test = tokenizer.texts_to_sequences(test_data[1])
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (30177, 128)
Shape of data tensor: (4269, 128)
Shape of data tensor: (4269, 128)


In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8990049056512643612
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15667560448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16819702324820167516
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-03-12 11:02:56.230926: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-12 11:02:56.289242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 11:02:56.290333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 11:02:56.291035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [12]:
Y_train = pd.get_dummies(train_data[0]).values
print('Shape of label tensor:', Y_train.shape)

Y_val = pd.get_dummies(val_data[0]).values
print('Shape of label tensor:', Y_val.shape)

Shape of label tensor: (30177, 31)
Shape of label tensor: (4269, 31)


In [13]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
from sklearn.metrics import f1_score, accuracy_score
# def f1(ytrue, ypred):
#     return f1_score(get_predicted(np.array(ytrue)), get_predicted(np.array(ypred)), average = 'weighted')

In [14]:
from keras import backend as K
def f1(true, pred): #shapes (batch, 4)

    #for metrics include these two lines, for loss, don't include them
    #these are meant to round 'pred' to exactly zeros and ones
    #predLabels = K.argmax(pred, axis=-1)
    #pred = K.one_hot(predLabels, 4) 


    ground_positives = K.sum(true, axis=0)       # = TP + FN
    pred_positives = K.sum(pred, axis=0)         # = TP + FP
    true_positives = K.sum(true * pred, axis=0)  # = TP
        #all with shape (4,)

    precision = (true_positives + K.epsilon()) / (pred_positives + K.epsilon()) 
    recall = (true_positives + K.epsilon()) / (ground_positives + K.epsilon()) 
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #not sure if this last epsilon is necessary
        #matematically not, but maybe to avoid computational instability
        #still with shape (4,)

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives)
    weighted_f1 = K.sum(weighted_f1)


    return weighted_f1 #for metrics, return only 'weighted_f1

In [15]:
mc = ModelCheckpoint('model_tam_1.h5', monitor='val_f1', mode='max', save_best_only=True)
cb_list = mc

In [16]:
from tensorflow.keras import layers

In [17]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(layers.Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(31, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=4, batch_size=32, callbacks = cb_list)
# Final evaluation of the model
model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
# print(classification_report(test_labels,model_pred_test))
#print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(Y_train, model_pred_train)))
#print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(Y_val, model_pred_test)))

2022-03-12 11:03:01.451592: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 11:03:01.452441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 11:03:01.453072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 11:03:01.453952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 11:03:01.454586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 1/4
944/944 [==============================] - 1045s 1s/step - loss: 2.8302 - f1: 0.1213 - val_loss: 2.7021 - val_f1: 0.1465
Epoch 2/4
944/944 [==============================] - 1055s 1s/step - loss: 2.4188 - f1: 0.2039 - val_loss: 2.7032 - val_f1: 0.1725
Epoch 3/4
944/944 [==============================] - 1066s 1s/step - loss: 1.8631 - f1: 0.3407 - val_loss: 2.9832 - val_f1: 0.1878
Epoch 4/4
944/944 [==============================] - 1072s 1s/step - loss: 1.3695 - f1: 0.4885 - val_loss: 3.3703 - val_f1: 0.1941


In [18]:

model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
f1 = f1_score(get_predicted(Y_train), get_predicted(model_pred_train), average = 'weighted')
acc = accuracy_score(get_predicted(Y_train), get_predicted(model_pred_train))
print(f1, acc)

0.7585133765120862 0.7755575438247672


In [19]:
f1 = f1_score(get_predicted(Y_val), get_predicted(model_pred_test), average = 'macro')
f1

0.10737390640477854

In [20]:
acc = accuracy_score(get_predicted(Y_val), get_predicted(model_pred_test))
print(f1, acc)

0.10737390640477854 0.2410400562192551


In [21]:
from sklearn.metrics import classification_report
report = classification_report(get_predicted(Y_val), get_predicted(model_pred_test),output_dict=True)


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [22]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.105263,0.027778,0.043956,72.00000
1,0.170213,0.121212,0.141593,66.00000
2,0.222222,0.040816,0.068966,49.00000
3,0.000000,0.000000,0.000000,18.00000
4,1.000000,0.037037,0.071429,27.00000
5,0.127119,0.130435,0.128755,115.00000
6,0.208897,0.207692,0.208293,520.00000
7,0.097561,0.042553,0.059259,94.00000
8,0.033333,0.019608,0.024691,51.00000
9,0.236041,0.329787,0.275148,282.00000
